In [ ]:
!pip install vllm pandas ta numpy accelerate


In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
import ta
import json
import re
from datetime import datetime

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BASE_DIR = "/content/drive/My Drive/IE403"
INPUT_DIR = os.path.join(BASE_DIR, "data/alpha_input")
OUTPUT_DIR = os.path.join(BASE_DIR, "data/transformer_input") # Nơi lưu kết quả cuối cùng
os.makedirs(OUTPUT_DIR, exist_ok=True)

TARGET_TICKERS = ["VIC", "FPT", "BID", "VNM", "VJC"]

print("✅ Đã cấu hình đường dẫn.")

✅ Đã cấu hình đường dẫn.


In [ ]:
from vllm import LLM, SamplingParams


MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct-AWQ"

print(f"⏳ Đang tải model: {MODEL_ID}...")

llm = LLM(
    model=MODEL_ID,
    quantization="awq",
    dtype="half",
    gpu_memory_utilization=0.9,
    max_model_len=8192,
    enforce_eager=False,
    trust_remote_code=True
)

print("✅ Model đã sẵn sàng sinh Alpha!")

⏳ Đang tải model: Qwen/Qwen2.5-Coder-32B-Instruct-AWQ...
INFO 12-17 08:45:42 [utils.py:253] non-default args: {'trust_remote_code': True, 'dtype': 'half', 'seed': None, 'max_model_len': 8192, 'disable_log_stats': True, 'quantization': 'awq', 'model': 'Qwen/Qwen2.5-Coder-32B-Instruct-AWQ'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-17 08:46:02 [model.py:637] Resolved architecture: Qwen2ForCausalLM
INFO 12-17 08:46:02 [model.py:1750] Using max model len 8192
INFO 12-17 08:46:02 [awq_marlin.py:167] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
INFO 12-17 08:46:05 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Parse safetensors files:   0%|          | 0/5 [00:00<?, ?it/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

WARNING 12-17 08:46:14 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 12-17 08:50:03 [llm.py:343] Supported tasks: ['generate']
✅ Model đã sẵn sàng sinh Alpha!


In [ ]:
TARGET_TICKERS = ["FPT"]


In [ ]:
def generate_and_apply_alphas(ticker):
    input_path = os.path.join(INPUT_DIR, f"{ticker}_full_features.csv")



    df = pd.read_csv(input_path, parse_dates=['date'], index_col='date')
    print(f"\nĐang xử lý {ticker} (Shape: {df.shape})...")


    columns_list = [
        "open", "high", "low", "close", "volume", "returns", "vwap",
        "rsi", "sma_5", "sma_20", "bb_upper", "bb_lower", "volatility",
        "sentiment_score", "sentiment_diff", "sentiment_ma5"
    ]

    cols_desc = ", ".join([f"df['{c}']" for c in columns_list])

    prompt = f"""<|im_start|>system
Bạn là chuyên gia Alpha Mining (Khai thác tín hiệu chứng khoán).
Nhiệm vụ: Viết 5 công thức Python (Formulaic Alphas) để dự đoán giá tăng.

Dữ liệu đầu vào `df` có các cột sau:
{cols_desc}

YÊU CẦU:
1. **Sáng tạo:** Kết hợp Sentiment với Technicals.
   - Ví dụ: `df['sentiment_ma5'] * (df['close'] - df['sma_20'])`
   - Ví dụ: `(df['sentiment_score'] > 0) * df['volume']`
2. **Cú pháp:** Chỉ viết biểu thức Python hợp lệ (one-line expression). Dùng `np` cho numpy.
3. **Logic:** Giá trị Alpha cao -> Tín hiệu Mua.

OUTPUT JSON ONLY:
{{
    "alpha_1": "expression",
    "alpha_2": "expression",
    "alpha_3": "expression",
    "alpha_4": "expression",
    "alpha_5": "expression"
}}
<|im_end|>
<|im_start|>user
Viết 5 Alphas cho mã {ticker}.
OUTPUT JSON ONLY:<|im_end|>
<|im_start|>assistant
"""

    sampling_params = SamplingParams(temperature=0.7, max_tokens=1024)
    outputs = llm.generate([prompt], sampling_params)
    generated_text = outputs[0].outputs[0].text

    try:
        match = re.search(r'\{.*\}', generated_text, re.DOTALL)
        if match:
            alphas = json.loads(match.group(0))
            print(f"   -> LLM đã sinh {len(alphas)} công thức.")

            with open(os.path.join(OUTPUT_DIR, f"{ticker}_formulas.json"), 'w') as f:
                json.dump(alphas, f, indent=4)

            for name, formula in alphas.items():
                try:
                    df[name] = eval(formula)
                except Exception as e:
                    print(f"      Lỗi công thức {name}: {e}. Điền 0.")
                    df[name] = 0.0


            df = df.replace([np.inf, -np.inf], np.nan).dropna()

            save_path = os.path.join(OUTPUT_DIR, f"{ticker}_final_dataset.csv")
            df.to_csv(save_path)
            print(f" Xong tại: {save_path}")
            print(f"Các cột mới: {list(alphas.keys())}")

        else:
            print("Lỗi: LLM không trả về JSON.")
            print(generated_text)

    except Exception as e:
        print(f"Lỗi xử lý: {e}")

for t in TARGET_TICKERS:
    generate_and_apply_alphas(t)


Đang xử lý FPT (Shape: (957, 16))...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

   -> LLM đã sinh 5 công thức.
 Xong tại: /content/drive/My Drive/IE403/data/transformer_input/FPT_final_dataset.csv
Các cột mới: ['alpha_1', 'alpha_2', 'alpha_3', 'alpha_4', 'alpha_5']


NameError: name 'df' is not defined